# Introduction
After the work I've done fixing the mass and charge balance, I will try to check if there are any fluxes with bounds at the extremes (+/- 1000). As a starting point of which reactions have unbounded flux, I start with the output from the  memote report found at '../reports/2020-05-04-4a1e905.html'

In [1]:
import cameo
import pandas as pd
import cobra.io
from cobra import Reaction

In [3]:
model = cobra.io.read_sbml_model('../model/p-thermo.xml')

In [4]:
solution = model.optimize()
solution.objective_value

1.6890391852957247

Memote prints out a list of reactions with ubounded flux in metabolism, indicating they are somehow involved in a cycle. Here I will try to look at those reactions.

I will look at what fluxes are associated to these reactions and then find fluxes that correspond well together and investigate how this is a cycle and why. And then also fix them where possible.

In [5]:
unbound = ["PTAr","GLUR","ACKr","MDH","ALAR","SUCOAS","UAG2EMA","UAG2E","G3PD1","G3PD2","G3PD","DHFR2i","DHFR","THFOR","ALATA_D","BNOR","BTCOAOR","ACOAD1f","MDH4","HPYRRx","GLYCOR_nadp","MTHFC","HACD1","HBCO_nadp","DHFOR","DHFOR2","ACMANAHY","ALCD4","ALCD4y","GARFT","FGFTh","HBCOAOR","HBCOAH","MANNHY","M_6DGHY","STACHY","STACHY2","H2MB4POR","DMPPOR","EAR60y","SUCOAACTr","HPI","HOXPRm","ACOAD1","ECOAH1","ASN__Lt","ALATA_L","FADRx","SERt","SERt2","ASNt","HACPOR"]

In [6]:
#wrong direction
model.reactions.NODOx.bounds = (-1000,0)

G3PD, G3PD1 and G3PD2: same reaction just with different co-factors. All three reactions are irreversible, so we just get generation of reducing equivalents. This needs to be fixed.
The G3PD reaction is wrong in kegg, it should be associated to a quinone instead of fad, and irreverisible. 

G3PD2 and G3PD1, we can leave both but then need to constrain the reaction direction of one of the two. As here we have no data as to which co-factor is used, we will leave all reactions but constrain them in the direction of the favored product. 

In [7]:
model.reactions.G3PD.add_metabolites({
    model.metabolites.fad_c:1,
    model.metabolites.fadh2_c:-1,
    model.metabolites.qh2_c:1,
    model.metabolites.mqn_c:-1
})

In [8]:
model.reactions.G3PD.bounds = (0,1000)

In [9]:
model.reactions.G3PD1.bounds = (0,1000)

In [10]:
model.reactions.G3PD2.bounds = (0,1000)

I've observed a cycle involved in the conversion of the stys_s and 6dg_c metabolites (rct STACHY, STACHY2, M_6DGHY and MANNHY). These reaction constitue polymerizations and hydrolysis. I think the polymerizations should use the NDP-sugars instead of just the sugars (in a similar way that we have captured carbohydrates in our biomass reaction). This is because polymerization requires energy input and this is the way we decided to capture it in biomass so it should also be considered here. 

So i will ensure that STACHY2 should use UDP-galactose instead of galactose. This should solve this cycle. 

In [12]:
model.reactions.STACHY2.add_metabolites({
    model.metabolites.gal_c:1,
    model.metabolites.udpgal_c:-1, 
    model.metabolites.udp_c:1,
    model.metabolites.h2o_c:-1
})

In [13]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

ALCD4 and ALCD4y are together forming a cycle. 

In [14]:
model.reactions.ALCD4.bounds = (-1000,0)

DHFOR vs DHFOR2: experimental evidence for B. subtilis and G. stearothermophilus shows predominant cofactor is NADPH. So remove DHFOR.
Same goes for DHFR vs DHFR2i: evidence for NADPH utilization, so remove the DHFR2i reaction.

In [15]:
model.remove_reactions(model.reactions.DHFOR)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [16]:
model.remove_reactions(model.reactions.DHFR2i)

Cycle between UAG2E, UAG2EMA and ACMANAHY: this needs to be fixed. ACMANAHY is in the wrong direction as it is a hydrolysis reaction. This should solve the cycle.

In [17]:
model.reactions.ACMANAHY.bounds = (-1000,0)

SERt and SERt2 are creating a cycle. Remove SERt2, make SERt reversible makes the most sense here.
Note: changing this really impacts biomass accumulation. It is still oke, but it severly decreases it, as this cycle can result in extra h_e production that can be used to make energy.

In [18]:
model.remove_reactions(model.reactions.SERt2)

In [19]:
model.reactions.SERt.bounds = (-1000,1000)

HACD1 and HBCO_nadp are the same reactions but with different cofactors. As both are irreversible you just get a cycle formed here. The HBCO_nadp seems to be an incorrect annotation, bacilli (and many bacteria in general)  have the NADH dependent version. There is a change our strain has the nadp version, but as this is causing problems for now we will do with the knowledge we have and remove the HBCO_nadp reaction. 

In [20]:
model.remove_reactions(model.reactions.HBCO_nadp)

ASNt ad ASN__Lt are duplicate reactions. remove the ASN__Lt reaction as this doesn't match the general ID consensus of our model.

In [21]:
model.remove_reactions(model.reactions.ASN__Lt)

ALATA_L, ALATA_D, GLUR, ALAR: The ALATA reactions are (in bacilli) used to donate an amino group from Alanine. Especially for the D-alanine. So I will fix the direction of the ALATA_D reaction in the direction of glutamate.

In [22]:
model.reactions.ALATA_D.bounds = (0,1000)

BNOR and BTCOAOR: two reactions are the same but with different co-factors and one is irreversible. The cofactor use is unknown here, but the general reaction is in the direction of the aldehyde dehydrogenase and so i will fix the direction of the BTCOAOR reaction.

In [23]:
model.reactions.BTCOAOR.bounds = (-1000,0)

HPYRRx and GLYCOR_nadp: both and the same reaction just with a different cofactor. The GLYCOR_nadp is irreverisble which is causing the cycle here. It should be in the same direction asthe HPYRRx reaction based on thermodynamics. Also the ID of this reaction can be made more compliant.

In [24]:
model.reactions.GLYCOR_nadp.id = 'HPYRRy'

In [25]:
model.reactions.HPYRRy.bounds = (-1000,0)

In [26]:
model.reactions.HACPOR.bounds = (-1000,0)

HPI, HOXPRm, HPYRRy and HPYRRx: HOXPRm should be fixed in one direction to prevent this cycle from cocuring. Based on thermodynamics we fix it in one direction.

In [27]:
model.reactions.HOXPRm.bounds = (-1000,0)

H2MB4POR and DMPPOR: same reaction with different cofactors in opposite directions.

In [28]:
model.reactions.DMPPOR.id = 'DMPPS'

In [29]:
model.reactions.DMPPS.bounds = (-1000,0)

DHFR, DHFR2 and THFOR: is a cycle of DHF to THF to folate, just cycling NADP(H) in the process. It seems the THFOR is in the wrong direction.

In [30]:
model.reactions.THFOR.bounds = (-1000,0)

ACOAD1f is in the wrong direction forming a cycle with ACOAD1.

In [31]:
model.reactions.ACOAD1f.bounds = (0,1000)

SUCOAS is the energy generating reaction in the TCA cycle. So need to change the bounds.
It seems that the SUCOAACTr is a wrong annotaion, looking back I cannot find any indication that this should be in the model. So I will remove it.

In [32]:
model.reactions.SUCOAS.bounds = (-1000,1000)

In [33]:
model.remove_reactions(model.reactions.SUCOAACTr)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



FGFTh and GARFT have the same ec code annotation but other KEGG annotations. In Brenda, this ec code matches with the GARFT reaction. In some organisms this enzyme has been shown to react with methf_c, but not in our strain or any Bacilli. Even if this would be the case, the predominant reaction would be GARFT, unless in the presence of methf in the medium. As we do not have an indication or strain can grow on this, I will remove the FGFTh reaction.

In [34]:
model.remove_reactions(model.reactions.FGFTh)

ALATA_L vs ALATA_D: the presence of both reactions causes an issue as we just get a cycle here. Looking into the enzyme (ec 2.6.1.29), we see that the prefered substrate is ALATA_L, with D-alanine being a competitive inhibitor of the enzyme in some organisms. As there isn't a reason to believe this enzyme should be so predominalty present, I will remove the ALATA_D reaction. the ala__D can still be made, just through the L metabolite. This doesn't cost any extra energy or other metabolites, soin the end the same can still be possible in the model.

In [35]:
model.remove_reactions(model.reactions.ALATA_L)

In [36]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')

Memote only gives the reactions that reach the maximum flux of 1000. But it may still be good to check there are no reactions that reach above a specific threshold, as this may also be something worth investigating. 

In the code below, I've checked that there are no reactions with fluxes above 50 or below -50 when only 10 mmol/gcdw h glucose is supplied (i.e. default medium), except the ETC/ATP synthase, which would be expected. So this looks good.

In [39]:
from cobra.flux_analysis import flux_variability_analysis

In [40]:
for rct in model.reactions:
    fva = flux_variability_analysis(model,rct)
    if fva['maximum'][0] > 50 or fva['minimum'][0] < -50 :
        print (fva)

           minimum     maximum
ATPS4r  102.623845  102.623845
          minimum    maximum
NADHDH  69.508037  69.508037
          minimum    maximum
CYTBO3  69.508106  69.508106


Finally I will remove the unconnected metabolites that are still hanging around. It's a bit wierd why this didn't delete before.

In [41]:
unconnected = ["selmethtrna_c","RNA_c","dna_c","amba_c","bcfa_c","g6p_B_c","qh2_e","nadp_e","acetone_c","4ampmp_c"]

In [42]:
unconnected_mets = []
for met in model.metabolites:
    if met.id in unconnected:
        unconnected_mets.append(met)

In [44]:
model.remove_metabolites(unconnected_mets)

In [45]:
#save&commit
cobra.io.write_sbml_model(model,'../model/p-thermo.xml')